In [ ]:
import pandas as pd
from itertools import product
import gurobipy as gp # Installation and license for Gurobi is required for the execution of optimization model
from gurobipy import GRB 

In [ ]:
# Step 1a: Data processing for 1-hub scenarios
# 'Distance_matrix_1hub.csv' is an ixj distance matrix exported from 'Distance computing.ipynb'; i represents the number of participants and j denotes the number of alternative conference hubs
Distance_matrix_1hub = pd.read_csv('Distance_matrix_1hub.csv')
Distance_matrix_1hub = Distance_matrix_1hub.set_index('Unnamed: 0')

airport_rank = Distance_matrix_1hub.columns.values

num_airport_1hub = 30
num_participant = 383
cartesian_prod_1hub = list(product(range(num_participant), range(num_airport_1hub)))
Distance_matrix_1hub = {(i,j): Distance_matrix_1hub.iloc[i,j] for i, j in cartesian_prod_1hub}

In [ ]:
# Step 1b: Data processing for multi-hub scenarios
# 'Distance_matrix_multihub.csv' is an ixj distance matrix exported from 'Distance computing.ipynb'; i represents the number of participants and j denotes the number of alternative conference hubs
Distance_matrix_multihub = pd.read_csv('Distance_matrix_multihub.csv')
Distance_matrix_multihub = Distance_matrix_multihub.set_index('Unnamed: 0')

airport_rank = Distance_matrix_multihub.columns.values

num_airport_multihub = 36
num_participant = 383
cartesian_prod_multihub = list(product(range(num_participant), range(num_airport_multihub)))
Distance_matrix_multihub = {(i,j): Distance_matrix_multihub.iloc[i,j] for i, j in cartesian_prod_multihub}

In [ ]:
# Step 1c: Data processing for "maximum travel distance" scenarios, additional to Step 1a/1b
# Dist_threshold: 1000, 3000, 5000, 10000 km
Dist_threshold = 1000
for key, value in Distance_matrix_1hub.items():
    if Distance_matrix_1hub[key] > Dist_threshold:
        Distance_matrix_1hub[key] = 1e8
        
for key, value in Distance_matrix_multihub.items():
    if Distance_matrix_multihub[key] > Dist_threshold:
        Distance_matrix_multihub[key] = 1e8        

In [ ]:
# Step 2a: Optimization model for 1-hub in-person and "maximum travel distance" scenarios
m = gp.Model('participant_airport')
# big M
M = 1e8
# number of hubs: can be any interger in [1,6]
N = 1

# define variable y[j]: Constraint (10) in manuscript
select_airport = m.addVars(num_airport_1hub, vtype=GRB.BINARY, name='select_airport')
# define variable x[i,j]: # Constraint (9) in manuscript
assign_participant = m.addVars(cartesian_prod_1hub, vtype=GRB.BINARY, name='assign_participant')

# Constraint (5) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for j in range(num_airport_1hub)) == 1 for i in range(num_participant)), name='Eq1')
# Constraint (6) in manuscript 
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) <= M*select_airport[j] for j in range(num_airport_1hub)), name='Eq2')
# Constraint (7) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) >= 15*select_airport[j] for j in range(num_airport_1hub)), name='Eq3')
# Constraint (8) in manuscript
m.addConstr((gp.quicksum(select_airport[j] for j in range(num_airport_1hub)) == N), name='Eq4')
# Equation (4) in manuscript
m.setObjective(assign_participant.prod(Distance_matrix_1hub), GRB.MINIMIZE)

m.optimize()

In [ ]:
# Step 2b: Optimization model for 1-hub "maximum virtual participation" scenarios
m = gp.Model('participant_airport')
# big M
M = 1e8
# number of hubs: can be any interger in [1,6]
N = 1
# percentage of in-person participation (i.e., 10%, 30%, 50%, or 70%) 
alpha = 0.1

# define variable y[j]: Constraint (10) in manuscript
select_airport = m.addVars(num_airport_1hub, vtype=GRB.BINARY, name='select_airport')
# define variable x[i,j]: # Constraint (9) in manuscript
assign_participant = m.addVars(cartesian_prod_1hub, vtype=GRB.BINARY, name='assign_participant')

# Constraint (12) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for j in range(num_airport_1hub)) <= 1 for i in range(num_participant)), name='Eq1')
# Constraint (6) in manuscript 
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) <= M*select_airport[j] for j in range(num_airport_1hub)), name='Eq2')
# Constraint (7) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) >= 15*select_airport[j] for j in range(num_airport_1hub)), name='Eq3')
# Constraint (8) in manuscript
m.addConstr((gp.quicksum(select_airport[j] for j in range(num_airport_1hub)) == N), name='Eq4')
# Constraint (13) in manuscript
m.addConstr((gp.quicksum(assign_participant[(i,j)] for i,j in cartesian_prod_1hub) >= alpha*num_participant), name='Eq5')
# Equation (4) in manuscript
m.setObjective(assign_participant.prod(Distance_matrix_1hub), GRB.MINIMIZE)

m.optimize()

In [ ]:
# Step 2c: Optimization model for multi-hub in-person and "maximum travel distance" scenarios
m = gp.Model('participant_airport')
# big M
M = 1e8
# number of hubs: can be any interger in [1,6]
N = 1

# define variable y[j]: Constraint (10) in manuscript
select_airport = m.addVars(num_airport_multihub, vtype=GRB.BINARY, name='select_airport')
# define variable x[i,j]: # Constraint (9) in manuscript
assign_participant = m.addVars(cartesian_prod_multihub, vtype=GRB.BINARY, name='assign_participant')

# Constraint (5) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for j in range(num_airport_multihub)) == 1 for i in range(num_participant)), name='Eq1')
# Constraint (6) in manuscript 
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) <= M*select_airport[j] for j in range(num_airport_multihub)), name='Eq2')
# Constraint (7) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) >= 15*select_airport[j] for j in range(num_airport_multihub)), name='Eq3')
# Constraint (8) in manuscript
m.addConstr((gp.quicksum(select_airport[j] for j in range(num_airport_multihub)) == N), name='Eq4')
# Equation (4) in manuscript
m.setObjective(assign_participant.prod(Distance_matrix_multihub), GRB.MINIMIZE)

m.optimize()

In [ ]:
# Step 2d: Optimization model for multi-hub in-person and "maximum travel distance" scenarios
m = gp.Model('participant_airport')
# big M
M = 1e8
# number of hubs: can be any interger in [1,6]
N = 1
# percentage of in-person participation (i.e., 10%, 30%, 50%, or 70%) 
alpha = 0.1

# define variable y[j]: Constraint (10) in manuscript
select_airport = m.addVars(num_airport_multihub, vtype=GRB.BINARY, name='select_airport')
# define variable x[i,j]: # Constraint (9) in manuscript
assign_participant = m.addVars(cartesian_prod_multihub, vtype=GRB.BINARY, name='assign_participant')

# Constraint (12) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for j in range(num_airport_multihub)) <= 1 for i in range(num_participant)), name='Eq1')
# Constraint (6) in manuscript 
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) <= M*select_airport[j] for j in range(num_airport_multihub)), name='Eq2')
# Constraint (7) in manuscript
m.addConstrs((gp.quicksum(assign_participant[(i,j)] for i in range(num_participant)) >= 15*select_airport[j] for j in range(num_airport_multihub)), name='Eq3')
# Constraint (8) in manuscript
m.addConstr((gp.quicksum(select_airport[j] for j in range(num_airport_multihub)) == N), name='Eq4')
# Constraint (13) in manuscript
m.addConstr((gp.quicksum(assign_participant[(i,j)] for i,j in cartesian_prod_multihub) >= alpha*num_participant), name='Eq5')
# Equation (4) in manuscript
m.setObjective(assign_participant.prod(Distance_matrix_multihub), GRB.MINIMIZE)

m.optimize()

In [ ]:
# print optimization results for the assignment of each participant to their optimized conference hubs
df_result = Distance_matrix_multihub.copy()
df_result = 0
df_result = {(i,j): abs(assign_participant[i, j].x) for i, j in assign_participant.keys()}
df_result_linear = [None] * num_participant
k=0
for i,j in assign_participant.keys():
    if (abs(assign_participant[i,j].x) > 0.99):
        df_result_linear[i] = int(airport_rank[j])+1
        k=k+1

df_result_final = pd.DataFrame(df_result_linear)
print(df_result_final)